## **Dynamic Capabilities**


In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
dbutils.widgets.text("file_name","")

In [0]:
p_file_name =dbutils.widgets.get("file_name")

In [0]:
p_file_name

### **Data Reading**

In [0]:
df = spark.read.format("csv").load("abfss://source@medicarestorageacct.dfs.core.windows.net/enrollment/medicare_enrollment")
df.display()

In [0]:
schema = StructType([
    StructField("State", StringType(), True),
    StructField("Enrollment_Total", IntegerType(), True),
    StructField("Enrollment_PartA", IntegerType(), True),
    StructField("Enrollment_PartB", IntegerType(), True),
    StructField("Enrollment_Other", IntegerType(), True)
])

In [0]:
df = spark.readStream.format("cloudfiles")\
            .option("cloudfiles.format", "csv")\
            .option("header", "true")\
            .option("nullable", "true")\
            .option("cloudFiles.schemaLocation", f"abfss://bronze@medicarestorageacct.dfs.core.windows.net/checkpoint_{p_file_name}")\
            .schema(schema)\
            .load(f"abfss://source@medicarestorageacct.dfs.core.windows.net/{p_file_name}")

## **Data Writing**

In [0]:
df.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://bronze@medicarestorageacct.dfs.core.windows.net/checkpoint_{p_file_name}") \
        .trigger(once=True)\
    .start("abfss://bronze@medicarestorageacct.dfs.core.windows.net/{p_file_name}")

In [0]:
df = spark.read.format("delta")\
            .load(f"abfss://bronze@medicarestorageacct.dfs.core.windows.net/{p_file_name}")
display(df)